# 셀레니움으로 자료 가져오기
* chrome 웹드라이버설치 [설치사이트]( https://chromedriver.chromium.org/downloads)
* 셀레니움참고 [참고사이트](https://galid1.tistory.com/478)

In [25]:
# #선행설치
# !pip install selenium

# !conda install -c conda-forge poppler #pdftotext 선행설치
# !pip install pdftotext

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

import time
import pdftotext # pdf 읽는용도
import pikepdf #lock걸려있는 pdf해제
import fitz # PyMuPDF

import pandas as pd
import numpy as np

import os
import re

# 본문내용긁어오기

In [9]:
#1회 사이클 이후 느려지는 경향이 있어 1회 약 10호 반복후 저장하는 형태로 변경 -> 다시 초기화(여기서하면 1ho씩 저장됨)

#저장위치
final_dict = dict()
final_dict['ho'] = []
final_dict['dates'] = []
final_dict['category'] = []
final_dict['city'] = []
final_dict['title'] = []
final_dict['contents'] = []

def get_contents():    

    try :
        for j in range(2,30):
            one_title = driver.find_element_by_xpath('//*[@id="container_suite"]/div[4]/div/div/ul/li[{num}]/div[1]/h2/a'.format(num =j))
            one_title.click()

            article_xpath = driver.find_element_by_xpath('//*[@id="center"]/article')

            #facebook, Twitter의 위치가 계속 변동되어 지우고 위치지정
            article = article_xpath.text.split('\n')
            article.remove('Facebook')
            article.remove('Twitter')

            #추출
            title = article[0]
            dates = article[2]
            ho = article[6]
            category = article[8]
            contents = article[9:-1]

            city = title.split(" ")[-1]
            if ')' in city:
                city = city.replace('(',"").replace(')',"")
            else :
                city = np.nan

            #저장 
            final_dict.get('dates',0).append(dates)
            final_dict.get('ho',0).append(ho)
            final_dict.get('category',0).append(category)
            final_dict.get('city',0).append(city)
            final_dict.get('title',0).append(title)
            final_dict.get('contents',0).append(contents)

            #뒤로가기
            time.sleep(0.5)
            driver.back()
    
    except Exception as e:
        pass 


In [16]:
def save_reset(final_dict):

    #csv 저장
    final_df = pd.DataFrame(final_dict)

    start_ho = final_df.ho.iloc[0]
    end_ho = final_df.ho.iloc[-1]

    final_df.to_csv('All_document({s}-{e}).csv'.format(s = start_ho, e = end_ho))
    
    #변수초기화
    final_dict = dict()
    final_dict['ho'] = []
    final_dict['dates'] = []
    final_dict['category'] = []
    final_dict['city'] = []
    final_dict['title'] = []
    final_dict['contents'] = []

# 과정통합

In [28]:
#전체 진행 (오류나면서 끝나는것이 맞다)
os.chdir('D:/Python/SDI 셀레니움/')
options = Options()
# chrome을 전체화면으로 넓히는 옵션입니다.
options.add_argument('--start-maximized') 
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#사이트이동
driver.get('http://www.sdi.re.kr')
time.sleep(2)

#세계도시동향 클릭
donghyang = driver.find_element_by_xpath('//*[@id="block-system-main-menu"]/div/ul/li[2]/a')
donghyang.click()
time.sleep(2)

#1페이지 내 다운로드 
for i in range(1,10):
    #처음페이지는 9개인데 이후 10개씩
    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
    ho.click()
    time.sleep(2)
    get_contents()
# 2페이지로 넘어가는게 아래의 숫자와 같고 9가 반복됨 (원인은 숫자가 계속 밀리기 때문)
#해당페이지는 2,3,4,5,6,7,
for page_num in [2,5,6,7,8,9]:
# for page_num in [2]:
    next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[{page_num}]/a'.format(page_num = page_num))
    next_page.click()
    time.sleep(2)
    try :
        for i in range(1,11):
        #처음페이지는 9개인데 이후 10개씩
            ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
            ho.click()
            time.sleep(2)
            get_contents()
    except:
        pass
#이후 반복 
while True:
    try:
        #이후반복
        while True:
            next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[9]/a')
            next_page.click()
            driver.implicitly_wait(10)
            for i in range(1,11):
                try:
            #처음페이지는 9개인데 이후 10개씩
                    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
                    ho.click()
                    time.sleep(1)
                    get_contents()
                except:
                    pass
    except:
        #에러난 페이지 내 다운로드 
        ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[2]/ul/li[1]/a')
        ho.click()
        for i in range(1,11):
            try:
                #처음페이지는 9개인데 이후 10개씩
                ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
                ho.click()
                time.sleep(1)
                get_contents()
            except: 
                pass
        pass


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: unhandled inspector error: {"code":-32000,"message":"Inspected target navigated or closed"}
  (Session info: chrome=81.0.4044.129)


### 특정페이지 이후(분할작업) 40페이지 이후

In [19]:
#전체 진행 (오류나면서 끝나는것이 맞다)
os.chdir('D:/Python/SDI 셀레니움/')
options = Options()
# chrome을 전체화면으로/ headless
options.add_argument('--start-maximized')
options.add_argument('--headless')
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#사이트이동
driver.get('https://www.si.re.kr/world_trends/22383?paged=41')
time.sleep(2)

#1페이지 내 다운로드 
for i in range(1,11):
    #처음페이지는 9개인데 이후 10개씩
    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
    ho.click()
    time.sleep(2)
    get_contents()
save_reset(final_dict)
#이후반복
while True:
    next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[9]/a')
    next_page.click()
    driver.implicitly_wait(10)
    for i in range(1,11):
        try:
    #처음페이지는 9개인데 이후 10개씩
            ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
            ho.click()
            time.sleep(1)
            get_contents()
        except:
            pass
    save_reset(final_dict)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container_suite"]/div[5]/ul/li[9]/a"}
  (Session info: headless chrome=81.0.4044.129)
